# Environment Setup

In [1]:
#note:  don't need all these libraries....can reduce
#
#       at one point i had a model developed in the codebase that was trying
#       to predict the race winner at the 25%, 50% and 75% point of the race
#       (turns out the laptime data has low information value to predict such an outcome)
#
import pyodbc
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from pandas.tseries.offsets import MonthEnd
from IPython.display import display
from dateutil.relativedelta import *
import matplotlib as mpl
import matplotlib.patches as mpatches
import re
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
import zipfile
import requests
import io
import time, datetime
from datetime import datetime, date, time, timedelta
import sys
sys.path.append("../py")
from utils import *
from pytimeparse.timeparse import timeparse
from ipywidgets import interact
#import qgrid
import ipywidgets as widgets
from ipywidgets import interact

pd.set_option('display.max.columns',None)
pd.set_option('display.max.rows',4000)
pd.options.display.max_seq_items = 4000

In [2]:
def getTime(x):
    return timeparse(x)

# Source Data

In [ ]:
# all data from alkamelsystems.com
#
# there are other datasets for each race that look like they may
# be of interest:  track weather, track conditions, etc
#
# somewhere in the results is a lap-by-lap flag condition....it's in pdf format only...you can
# adjust laps by hand coding the sector level yellows and then filter those laps out to conduct
# true performance comparison

In [3]:
#create imsa aggregate dataset
url_1 = 'http://fiawec.alkamelsystems.com/Results/11_2022/01_SEBRING/402_FIA%20WEC/202203181200_Race/Final%20Results/23_Analysis_Race_Hour%208.CSV'
url_2 = 'http://imsa.alkamelsystems.com/Results/22_2022/05_Sebring%20International%20Raceway/01_IMSA%20WeatherTech%20SportsCar%20Championship/202203191010_Race/12_Hour%2012/23_Time%20Cards_Race.CSV'
url_3 = 'http://imsa.alkamelsystems.com/Results/22_2022/02_Daytona%20International%20Speedway/01_IMSA%20WeatherTech%20SportsCar%20Championship/202201291340_Race/24_Hour%2024/23_Time%20Cards_Race.CSV'
url_4 = 'http://imsa.alkamelsystems.com/Results/22_2022/06_Long%20Beach%20Street%20Circuit/01_IMSA%20WeatherTech%20SportsCar%20Championship/202204091405_Race/23_Time%20Cards_Race.CSV'

imsa = pd.DataFrame()

file_loop = [['s12','2'],['d24','3'],['lbgp','4']]
for race in file_loop:
    foo = pd.read_csv(eval('url_'+race[1]), header=0, sep=';')
    foo['race'] = race[0]
    imsa = pd.concat([imsa, foo], axis=0, ignore_index=True)


In [4]:
imsa.head(50)

,NUMBER,DRIVER_NUMBER,LAP_NUMBER,LAP_TIME,LAP_IMPROVEMENT,CROSSING_FINISH_LINE_IN_PIT,S1,S1_IMPROVEMENT,S2,S2_IMPROVEMENT,S3,S3_IMPROVEMENT,KPH,ELAPSED,HOUR,S1_LARGE,S2_LARGE,S3_LARGE,TOP_SPEED,DRIVER_NAME,PIT_TIME,CLASS,GROUP,TEAM,MANUFACTURER,FLAG_AT_FL,Unnamed: 26,race
0,1,2,1,2:10.504,0,B,10.563,0,1:14.584,0,45.357,0,166.0,2:10.504,10:12:58.463,0:10.563,1:14.584,0:45.357,NaN,Sebastien Bourdais,NaN,DPi,NaN,Cadillac Racing,Cadillac,GF,NaN,s12
1,1,2,2,4:21.156,0,NaN,2:36.939,0,1:12.878,0,31.339,0,83.0,6:31.660,10:17:19.619,2:36.939,1:12.878,0:31.339,NaN,Sebastien Bourdais,0:02:39.527,DPi,NaN,Cadillac Racing,Cadillac,GF,NaN,s12
2,1,2,3,1:52.612,0,NaN,10.318,0,1:10.933,0,31.361,0,192.4,8:24.272,10:19:12.231,0:10.318,1:10.933,0:31.361,NaN,Sebastien Bourdais,NaN,DPi,NaN,Cadillac Racing,Cadillac,GF,NaN,s12
3,1,2,4,1:57.297,0,NaN,10.772,0,1:14.629,0,31.896,0,184.7,10:21.569,10:21:09.528,0:10.772,1:14.629,0:31.896,NaN,Sebastien Bourdais,NaN,DPi,NaN,Cadillac Racing,Cadillac,GF,NaN,s12
4,1,2,5,1:56.730,0,NaN,10.762,0,1:14.330,0,31.638,0,185.6,12:18.299,10:23:06.258,0:10.762,1:14.330,0:31.638,NaN,Sebastien Bourdais,NaN,DPi,NaN,Cadillac Racing,Cadillac,GF,NaN,s12
5,1,2,6,1:51.055,0,NaN,10.718,0,1:09.936,0,30.401,0,195.1,14:09.354,10:24:57.313,0:10.718,1:09.936,0:30.401,NaN,Sebastien Bourdais,NaN,DPi,NaN,Cadillac Racing,Cadillac,GF,NaN,s12
6,1,2,7,1:51.208,0,NaN,10.301,0,1:09.111,0,31.796,0,194.8,16:00.562,10:26:48.521,0:10.301,1:09.111,0:31.796,NaN,Sebastien Bourdais,NaN,DPi,NaN,Cadillac Racing,Cadillac,GF,NaN,s12
7,1,2,8,1:50.721,0,NaN,10.464,0,1:09.502,0,30.755,0,195.7,17:51.283,10:28:39.242,0:10.464,1:09.502,0:30.755,NaN,Sebastien Bourdais,NaN,DPi,NaN,Cadillac Racing,Cadillac,GF,NaN,s12
8,1,2,9,1:55.785,0,NaN,10.227,0,1:14.693,0,30.865,0,187.1,19:47.068,10:30:35.027,0:10.227,1:14.693,0:30.865,NaN,Sebastien Bourdais,NaN,DPi,NaN,Cadillac Racing,Cadillac,GF,NaN,s12
9,1,2,10,1:51.981,0,NaN,10.084,0,1:10.187,0,31.710,0,193.5,21:39.049,10:32:27.008,0:10.084,1:10.187,0:31.710,NaN,Sebastien Bourdais,NaN,DPi,NaN,Cadillac Racing,Cadillac,GF,NaN,s12


# Feature Engineering

In [5]:
   
def feature_engineering(df):
    #remove spaces from col names
    df.columns = [c.strip() for c in df.columns]
    
    #convert from kph to mph (not collected for imsa, but is for wec)
    df['TOP_SPEED_MPH'] = df['TOP_SPEED'] / 1.609344
    df['MPH'] = df.KPH / 1.609344
    
    #set up various identifiers
    df[['NUMBER','DRIVER_NUMBER']] = df[['NUMBER','DRIVER_NUMBER']].astype(str)
    df['CAR_DRIVER'] = df['NUMBER'] + '_' + df['DRIVER_NUMBER']
    df['CAR_TEAM'] = df['NUMBER'] + '_' + df['TEAM']

    #convert time from hh:mm:ss.sss to total seconds 
    df['LAP_TIME_S'] = df['LAP_TIME'].apply(getTime)
    mask = df.PIT_TIME.notna()
    df.loc[mask, 'PIT_TIME_S']=df[mask].apply(lambda row: getTime(row.PIT_TIME),axis=1)
    
    #Also set overall lap = 1 to be a driver change
    df['driverchange'] = np.where((~df['DRIVER_NUMBER'].eq(df['DRIVER_NUMBER'].shift())) | (df['LAP_NUMBER']==1), 1,0)
    df['DRIVER_SESSION'] = df.groupby(['NUMBER', 'DRIVER_NUMBER'])['driverchange'].cumsum().astype(int)
    df['INLAP'] = (df['CROSSING_FINISH_LINE_IN_PIT'] == 'B')
    
    #Make no assumptions about table order - so sort by lap number
    df = df.sort_values(['NUMBER', 'TEAM', 'LAP_NUMBER'])
    
    # Identify a new stint for each car by shifting the pitting / INLAP flag within car tables
    df['OUTLAP'] = df.groupby('NUMBER')['INLAP'].shift(fill_value=True)
    df['CAR_STINT'] = df.groupby('NUMBER')['OUTLAP'].cumsum().astype(int)
    
    #Driver stint - a cumulative count for each driver of their stints
    df['DRIVER_STINT'] = df.groupby('CAR_DRIVER')['OUTLAP'].cumsum().astype(int)
    
    #Let's also derive another identifier - CAR_DRIVER_STINT
    df['CAR_DRIVER_STINT'] = df['CAR_DRIVER'] + '_' + df['DRIVER_STINT'].astype(str)
    
    #Driver session stint - a count for each driver of their stints within a particular driving session
    df['DRIVER_SESSION_STINT'] = df.groupby(['CAR_DRIVER','DRIVER_SESSION'])['OUTLAP'].cumsum().astype(int)
    
    # lap count by car stint - that is, between each pit stop
    df['LAPS_CAR_STINT'] = df.groupby(['NUMBER','CAR_STINT']).cumcount()+1
    
    #lap count by driver
    df['LAPS_DRIVER'] = df.groupby('CAR_DRIVER').cumcount()+1
    
    #lap count by driver session
    df['LAPS_DRIVER_SESSION'] = df.groupby(['CAR_DRIVER','DRIVER_SESSION']).cumcount()+1
    
    #lap count by driver stint
    df['LAPS_DRIVER_STINT'] = df.groupby(['CAR_DRIVER','DRIVER_STINT']).cumcount()+1
    
    
    df['S1_LARGE'] = np.where(df.S1_LARGE.str.count(':') == 1
                              ,'0:'+df.S1_LARGE
                              ,df.S1_LARGE
                             )   
    df[['h','m', 's']] = df['S1_LARGE'].str.split(':', expand=True)
    df[['h','m','s']] = df[['h', 'm','s']].astype(float)
    df['s1_seconds'] = df.apply(lambda row: row['h']*60*60 + row['m']*60 + row['s'], axis=1)

    df['S2_LARGE'] = np.where(df.S2_LARGE.str.count(':') == 1
                              ,'0:'+df.S2_LARGE
                              ,df.S2_LARGE
                             )   
    df[['h','m', 's']] = df['S2_LARGE'].str.split(':', expand=True)
    df[['h','m','s']] = df[['h', 'm','s']].astype(float)
    df['s2_seconds'] = df.apply(lambda row: row['h']*60*60 + row['m']*60 + row['s'], axis=1)
    
    df['S3_LARGE'] = np.where(df.S3_LARGE.str.count(':') == 1
                              ,'0:'+df.S3_LARGE
                              ,df.S3_LARGE
                             )   
    df[['h','m', 's']] = df['S3_LARGE'].str.split(':', expand=True)
    df[['h','m','s']] = df[['h', 'm','s']].astype(float)
    df['s3_seconds'] = df.apply(lambda row: row['h']*60*60 + row['m']*60 + row['s'], axis=1)
            
    df['s1_cume'] = df['s1_seconds']
    df['s2_cume'] = df['s1_cume'] + df['s2_seconds']
    df['s3_cume'] = df['s2_cume'] + df['s3_seconds']
    return df

In [6]:
imsa = feature_engineering(imsa)


# Data Discovery

In [8]:
imsa['CAR_TEAM'].value_counts()

10_Konica Minolta Acura ARX-05            1185
31_Whelen Engineering Racing              1185
5_JDC Miller MotorSports                  1185
60_Meyer Shank Racing W/Curb-Agajanian    1185
2_Cadillac Racing                         1158
1_Cadillac Racing                         1102
14_VasserSullivan                         1100
52_PR1 Mathiasen Motorsports              1096
29_Racing Team Nederland                  1095
70_inception racing                       1094
3_Corvette Racing                         1090
48_Ally Cadillac                          1085
22_United Autosports                      1082
16_Wright Motorsports                     1081
79_WeatherTech Racing                     1064
18_Era Motorsport                         1062
9_Pfaff Motorsports                       1062
99_Team Hardpoint                         1059
33_Sean Creech Motorsport                 1053
66_Gradient Racing                        1033
54_CORE Autosport                         1029
62_Risi Compe

In [13]:
race = imsa[imsa.race == 'd24']

@interact(CAR_TEAM=race['CAR_TEAM'].unique().tolist(),)
def plotLapByNumber(CAR_TEAM):
    race[race['CAR_TEAM']==CAR_TEAM].plot(x='LAP_NUMBER', y='LAP_TIME_S')

interactive(children=(Dropdown(description='CAR_TEAM', options=('1_Cadillac Racing', '10_Konica Minolta Acura …

In [14]:
@interact(number=race['CAR_TEAM'].unique().tolist(),)
def plotLapByNumberDriver(number):
    # We can pivot long to wide on driver number,
    # then plot all cols against the lapnumber index
    race[race['CAR_TEAM']==number].pivot(index='LAP_NUMBER',
                                               columns='DRIVER_NAME',
                                               values='LAP_TIME_S').plot()

interactive(children=(Dropdown(description='number', options=('1_Cadillac Racing', '10_Konica Minolta Acura AR…

In [15]:
#this one has a bug in it

@interact(number=race['CAR_TEAM'].unique().tolist(),
          pitentrylap=True)
def plotLapByNumberDriverWithPit(number, pitentrylap):
    # We can pivot long to wide on driver number,
    # then plot all cols against the lapnumber index
    #Graph the matplotlib axes so we can overplot onto them
    ax = race[race['NUMBER']==number].pivot(index='LAP_NUMBER',
                                                    columns='DRIVER_NAME',
                                                    values='LAP_TIME_S').plot()
    # Also add in pit laps
    # Filter rows that identify both the car
    # and the laps on which the car crossed the finish line in the pit
    pitcondition = (race['CROSSING_FINISH_LINE_IN_PIT']=='B') if pitentrylap \
                    else ~(lbgp['PIT_TIME'].isnull())
    inpitlaps = race[(race['CAR_TEAM']==number) & (pitcondition) ]
    # Plot a marker for each of those rows
    inpitlaps.plot.scatter(x='LAP_NUMBER',y='LAP_TIME_S', ax=ax)
    

interactive(children=(Dropdown(description='number', options=('1_Cadillac Racing', '10_Konica Minolta Acura AR…

In [16]:
cars = widgets.Dropdown(
    options=race['NUMBER'].unique(), # value='1',
    description='Car:', disabled=False )

drivers = widgets.Dropdown(
    options=race[race['NUMBER']==cars.value]['CAR_DRIVER'].unique(),
    description='Driver:', disabled=False)

driversessions = widgets.Dropdown(
    options=race[race['CAR_DRIVER']==drivers.value]['DRIVER_SESSION'].unique(),
    description='Session:', disabled=False)

driverstints = widgets.Dropdown(
    options=race[race['DRIVER_SESSION']==driversessions.value]['DRIVER_SESSION_STINT'].unique(),
    description='Stint:', disabled=False)

def update_drivers(*args):
    driverlist = race[race['NUMBER']==cars.value]['CAR_DRIVER'].unique()
    drivers.options = driverlist
    
def update_driver_session(*args):
    driversessionlist = race[(race['CAR_DRIVER']==drivers.value)]['DRIVER_SESSION'].unique()
    driversessions.options = driversessionlist
    
def update_driver_stint(*args):
    driverstintlist = race[(race['CAR_DRIVER']==drivers.value) &
                               (race['DRIVER_SESSION']==driversessions.value)]['DRIVER_SESSION_STINT'].unique()
    driverstints.options = driverstintlist
    
cars.observe(update_drivers, 'value')
drivers.observe(update_driver_session,'value')
driversessions.observe(update_driver_stint,'value')

def laptime_table(car, driver, driversession, driverstint):
    #just basic for now...
    display(race[(race['CAR_DRIVER']==driver) &
                     (race['DRIVER_SESSION']==driversession) &
                     (race['DRIVER_SESSION_STINT']==driverstint) ][['CAR_DRIVER', 'DRIVER_SESSION',
                                                         'DRIVER_STINT', 'DRIVER_SESSION_STINT',
                                                         'LAP_NUMBER','LAP_TIME', 'LAP_TIME_S']])
    
interact(laptime_table,
         car=cars, driver=drivers,
         driversession=driversessions, driverstint=driverstints);


interactive(children=(Dropdown(description='Car:', options=('1', '10', '11', '12', '13', '14', '15', '16', '18…

# DPi vs LMDh

In [13]:
sebring1k['FLAG_AT_FL'].value_counts()

GF     6124
SF      176
FCY      62
RF       45
Name: FLAG_AT_FL, dtype: int64

In [50]:
dpi  = sebring12[(sebring12.CLASS == 'DPi')      & (sebring12['CROSSING_FINISH_LINE_IN_PIT'] != 'B') & (sebring12.FLAG_AT_FL.isin(['GF'])) & (sebring12.INLAP == False) & (sebring12.OUTLAP == False)]
lmdh = sebring1k[(sebring1k.CLASS == 'HYPERCAR') & (sebring1k['CROSSING_FINISH_LINE_IN_PIT'] != 'B') & (sebring1k.FLAG_AT_FL.isin(['GF'])) & (sebring1k.INLAP == False) & (sebring1k.OUTLAP == False)]
dpi_daytona = daytona24[(daytona24.CLASS == 'DPi') & (daytona24['CROSSING_FINISH_LINE_IN_PIT'] != 'B') & (daytona24.FLAG_AT_FL.isin(['GF'])) & (daytona24.INLAP == False) & (daytona24.OUTLAP == False)]

# driver changes by car

In [18]:
imsa[(imsa.driverchange == 1) & (imsa.CLASS == 'DPi')].groupby(['race','CAR_TEAM'])['driverchange'].count()

race  CAR_TEAM                              
d24   10_Konica Minolta Acura ARX-05            10
      1_Cadillac Racing                          8
      2_Cadillac Racing                         12
      31_Whelen Engineering Racing              12
      48_Ally Cadillac                          14
      5_JDC Miller MotorSports                  12
      60_Meyer Shank Racing W/Curb-Agajanian    14
lbgp  10_Konica Minolta Acura ARX-05             2
      1_Cadillac Racing                          2
      2_Cadillac Racing                          2
      31_Whelen Engineering Racing               3
      5_JDC Miller MotorSports                   2
      60_Meyer Shank Racing W/Curb-Agajanian     2
s12   10_Konica Minolta Acura ARX-05             7
      1_Cadillac Racing                          6
      2_Cadillac Racing                          8
      31_Whelen Engineering Racing              10
      48_Ally Cadillac                           9
      5_JDC Miller MotorSports       